In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import staintools
import random
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from tqdm import tqdm, tqdm_notebook
#tqdm_notebook().pandas()


#De-noising
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import img_as_ubyte, img_as_float
from skimage import io
from scipy import ndimage as nd
#Image Quality:
import imquality.brisque as brisque


#Threads
import concurrent.futures

#Time
import time

#Warnings
import warnings


import stain_utils as utils
import stainNorm_Reinhard
import stainNorm_Macenko
import stainNorm_Vahadane
%load_ext autoreload
%autoreload 2

%matplotlib inline
random.seed(123)

In [2]:
warnings.filterwarnings("ignore")

In [3]:
path = './LC25000/lung_colon_image_set/lung_image_sets/'
dir_lst=[]
for i in tqdm_notebook(os.listdir(path)):
    if os.path.isdir(path+i):
        dir_lst.append(i)
dir_lst

['lung_n', 'lung_scc', 'lung_aca']

In [4]:
def standardize_brightness(image):
    
    p = np.percentile(image,90)
    return np.clip(image * 255.0/p,0,255).astype(np.uint8)
    

In [5]:
dir_lst

['lung_n', 'lung_scc', 'lung_aca']

## Denoising Final

In [19]:
os.mkdir('denoise/')
for dirs in dir_lst:
    os.mkdir('denoise/'+dirs)

FileExistsError: [Errno 17] File exists: 'denoise/'

In [36]:
brisque_score_old = {'lung_aca': [],'lung_scc':[],'lung_n':[]}

In [47]:
#Safe_Mode
path='./LC25000/lung_colon_image_set/lung_image_sets/'
lung_n = [path+'lung_n'+'/'+image for image in os.listdir(path+'lung_n')]

def denoise_safe_mode(imgPath):
    img_f = img_as_float(io.imread(imgPath))
    sigma_est = np.mean(estimate_sigma(img_f, multichannel=True))
    patch_kw = dict(patch_size=3,      
                    patch_distance=2,
                    multichannel=True)
    denoise_img = denoise_nl_means(img_f, h=1.15 * sigma_est, fast_mode=False,
                                   **patch_kw)
    denoise_img_as_8byte = img_as_ubyte(denoise_img)
        
    a = imgPath.split('/')
    plt.imsave('./denoise/'+a[len(a)-2]+'/'+a[len(a)-1],denoise_img_as_8byte)

print(len(lung_n))
#print(lung_n[4995:5000])

5000


In [48]:
t1 = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(denoise_safe_mode, lung_n)
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Finished in 3348.211063697003 seconds


In [21]:
path='./LC25000/lung_colon_image_set/lung_image_sets/'
def denoise_image(dirs):
    for file in tqdm_notebook(os.listdir(path+dirs)):
        imgPath = path+dirs+'/'+file
        img_f = img_as_float(io.imread(imgPath))
        sigma_est = np.mean(estimate_sigma(img_f, multichannel=True))
        patch_kw = dict(patch_size=3,      
                patch_distance=2,  
                multichannel=True)
        denoise_img = denoise_nl_means(img_f, h=1.15 * sigma_est, fast_mode=False,
                           **patch_kw)
        denoise_img_as_8byte = img_as_ubyte(denoise_img)
        
        a = imgPath.split('/')
        plt.imsave('./denoise/'+a[len(a)-2]+'/'+a[len(a)-1],denoise_img_as_8byte)

In [22]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(denoise_image, dir_lst)

## Final code Stain Normalization

In [49]:
path = './denoise/'
for dirs in dir_lst:
    if dirs == 'lung_aca':
        lst_aca = [img for img in os.listdir(path+dirs)]
        random.seed(123)
        num_imgs_aca = len(lst_aca)
        tg_img_aca = lst_aca[random.randint(0,num_imgs_aca)]
        print((num_imgs_aca,tg_img_aca))
        target_aca = cv2.imread(path+dirs+'/'+tg_img_aca)
        target_aca = cv2.cvtColor(target_aca,cv2.COLOR_BGR2RGB)
        target_aca = standardize_brightness(target_aca)
        normalizer_aca = staintools.StainNormalizer(method='vahadane')
        normalizer_aca.fit(target_aca)
        
    elif dirs == 'lung_scc':
        lst_scc = [img for img in os.listdir(path+dirs)]
        random.seed(123)
        num_imgs_scc = len(lst_scc)
        tg_img_scc = lst_scc[random.randint(0,num_imgs_scc)]
        print((num_imgs_scc,tg_img_scc))
        target_scc = cv2.imread(path+dirs+'/'+tg_img_scc)
        target_scc = cv2.cvtColor(target_scc,cv2.COLOR_BGR2RGB)
        target_scc = standardize_brightness(target_scc)
        normalizer_scc = staintools.StainNormalizer(method='vahadane')
        normalizer_scc.fit(target_scc)
    else:
                    
        lst_n = [img for img in os.listdir(path+dirs)]
        random.seed(123)
        num_imgs_n = len(lst_n)
        tg_img_n = lst_n[random.randint(0,num_imgs_n)]
        print((num_imgs_n,tg_img_n))           
        target_n = cv2.imread(path+dirs+'/'+tg_img_n)
        target_n = cv2.cvtColor(target_n,cv2.COLOR_BGR2RGB)
        target_n = standardize_brightness(target_n)
        normalizer_n = staintools.StainNormalizer(method='vahadane')
        normalizer_n.fit(target_n)

print(path+dirs+'/'+tg_img_aca)
print(path+dirs+'/'+tg_img_scc)
print(path+dirs+'/'+tg_img_n)

(4998, 'lungn4637.jpeg')
(5000, 'lungscc2410.jpeg')
(5000, 'lungaca4491.jpeg')
./denoise/lung_aca/lungaca4491.jpeg
./denoise/lung_aca/lungscc2410.jpeg
./denoise/lung_aca/lungn4637.jpeg


In [51]:
os.mkdir('normalize/')
for dirs in dir_lst:
    os.mkdir('normalize/'+dirs)

In [ ]:
path = './denoise/'
def stain_Normalize(dirs):
    for file in tqdm_notebook(os.listdir(path+dirs)):
        imgPath = path+dirs+'/'+file
        img = cv2.cvtColor(cv2.imread(imgPath),cv2.COLOR_BGR2RGB)
        
        if dirs == 'lung_aca':
            n_img = normalizer_aca.transform(img)
            n_img = standardize_brightness(n_img)
        elif dirs == 'lung_scc':
            n_img = normalizer_scc.transform(img)
            n_img = standardize_brightness(n_img)
        else:
            n_img = normalizer_n.transform(img)
            n_img = standardize_brightness(n_img)
            
        #resize
        resize = cv2.resize(n_img,(500,500),interpolation = cv2.INTER_CUBIC)
        
        a = imgPath.split('/')
        plt.imsave('./normalize/'+a[len(a)-2]+'/'+a[len(a)-1],resize)

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(stain_Normalize, dir_lst)

## Normalizatoin Safe mode

In [6]:
path = './denoise/'

for dirs in dir_lst:
    if dirs == 'lung_aca':
        lst_aca = [img for img in os.listdir(path+dirs)]
        num_imgs_aca = len(lst_aca)
        tg_img_aca = lst_aca[random.randint(0,200)]
        print((num_imgs_aca,tg_img_aca))
        target_aca = cv2.imread(path+dirs+'/'+tg_img_aca)
        target_aca = cv2.cvtColor(target_aca,cv2.COLOR_BGR2RGB)
        target_aca = standardize_brightness(target_aca)
        normalizer_aca = staintools.StainNormalizer(method='vahadane')
        normalizer_aca.fit(target_aca)
        
    elif dirs == 'lung_scc':
        lst_scc = [img for img in os.listdir(path+dirs)]
        num_imgs_scc = len(lst_scc)
        tg_img_scc = lst_scc[random.randint(0,200)]
        print((num_imgs_scc,tg_img_scc))
        target_scc = cv2.imread(path+dirs+'/'+tg_img_scc)
        target_scc = cv2.cvtColor(target_scc,cv2.COLOR_BGR2RGB)
        target_scc = standardize_brightness(target_scc)
        normalizer_scc = staintools.StainNormalizer(method='vahadane')
        normalizer_scc.fit(target_scc)
    else:
                    
        lst_n = [img for img in os.listdir(path+dirs)]
        num_imgs_n = len(lst_n)
        tg_img_n = lst_n[random.randint(0,200)]
        print((num_imgs_n,tg_img_n))           
        target_n = cv2.imread(path+dirs+'/'+tg_img_n)
        target_n = cv2.cvtColor(target_n,cv2.COLOR_BGR2RGB)
        target_n = standardize_brightness(target_n)
        normalizer_n = staintools.StainNormalizer(method='vahadane')
        normalizer_n.fit(target_n)

print(path+dirs+'/'+tg_img_aca)
print(path+dirs+'/'+tg_img_scc)
print(path+dirs+'/'+tg_img_n)

(4998, 'lungn719.jpeg')
(5000, 'lungscc3329.jpeg')
(5000, 'lungaca1443.jpeg')
./denoise/lung_aca/lungaca1443.jpeg
./denoise/lung_aca/lungscc3329.jpeg
./denoise/lung_aca/lungn719.jpeg


In [1]:
path = './denoise2/'
lung_n = [path+'lung_n'+'/'+image for image in os.listdir(path+'lung_n')]
lung_aca = [path+'lung_aca'+'/'+image for image in os.listdir(path+'lung_aca')]
lung_scc = [path+'lung_scc'+'/'+image for image in os.listdir(path+'lung_scc')]

def stain_Normalize_safeMode(imgPath):
    
    img = cv2.cvtColor(cv2.imread(imgPath),cv2.COLOR_BGR2RGB)

    if dirs == 'lung_aca':
        n_img = normalizer_aca.transform(img)
        n_img = standardize_brightness(n_img)
    elif dirs == 'lung_scc':
        n_img = normalizer_scc.transform(img)
        n_img = standardize_brightness(n_img)
    else:
        n_img = normalizer_n.transform(img)
        n_img = standardize_brightness(n_img)

    #resize
    resize = cv2.resize(n_img,(500,500),interpolation = cv2.INTER_CUBIC)

    a = imgPath.split('/')
    plt.imsave('./normalize/'+a[len(a)-2]+'/'+a[len(a)-1],resize)



NameError: name 'os' is not defined

In [10]:
t1 = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(stain_Normalize_safeMode, lung_n)
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Finished in 9345.374173839002 seconds


In [8]:
t1 = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(stain_Normalize_safeMode, lung_aca)
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Finished in 8183.0972539780005 seconds


In [9]:
t1 = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(stain_Normalize_safeMode, lung_scc)
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Finished in 8342.126284139002 seconds
